# import the relevent packages

In [2]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

# Data 

In [3]:
mnist_dataset,mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']

num_validation_samples = 0.1 *mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = 0.1 *mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image,label):
  image = tf.cast(image,tf.float32)
  image /=255.
  return image,label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000
shuffled_train_and_validation_data  = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data =validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs,validation_targets = next(iter(validation_data))


# Model
# Outline the model

In [5]:
input_size =784
output_size =10
hidden_layer_size =100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape =(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax'),
])

# Choose the optimizer and the loss function

In [6]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'] )

# Training

In [7]:
NUM_EPOCHS = 5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 9s - loss: 0.3333 - accuracy: 0.9037 - val_loss: 0.1724 - val_accuracy: 0.9498 - 9s/epoch - 17ms/step
Epoch 2/5
540/540 - 5s - loss: 0.1391 - accuracy: 0.9590 - val_loss: 0.1079 - val_accuracy: 0.9690 - 5s/epoch - 10ms/step
Epoch 3/5
540/540 - 5s - loss: 0.0979 - accuracy: 0.9703 - val_loss: 0.0882 - val_accuracy: 0.9742 - 5s/epoch - 10ms/step
Epoch 4/5
540/540 - 5s - loss: 0.0741 - accuracy: 0.9774 - val_loss: 0.0701 - val_accuracy: 0.9793 - 5s/epoch - 9ms/step
Epoch 5/5
540/540 - 5s - loss: 0.0610 - accuracy: 0.9812 - val_loss: 0.0610 - val_accuracy: 0.9813 - 5s/epoch - 8ms/step


# Test The model

In [8]:
test_loss, test_accuracy = model.evaluate(test_data)

10/10 [==============================] - 1s 137ms/step - loss: 0.0832 - accuracy: 0.9761


In [9]:
print('Test loss : {0:.2f}. test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100))

Test loss : 0.08. test accuracy: 97.61%
